In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from scipy.optimize import curve_fit
import h5py
import pyvisa
import time
from scipy.signal import savgol_filter
import scipy.signal as signal
from scipy import interpolate
from matplotlib.ticker import MaxNLocator
from scipy.optimize import least_squares

In [2]:
factor = 2
SMALL_SIZE = factor*8
MEDIUM_SIZE = factor*10
BIGGER_SIZE = factor*12
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
rm = pyvisa.ResourceManager()
rm.list_resources()

('ASRL4::INSTR',
 'ASRL5::INSTR',
 'ASRL6::INSTR',
 'ASRL8::INSTR',
 'ASRL11::INSTR')

In [6]:
for i in range(2):
    try:
        HV_control = rm.open_resource('ASRL11::INSTR')
        print(HV_control.query("?"))
    except pyvisa.VisaIOError:
        pass

HV_control v1.3 ready. Board B.



In [5]:
DMM = rm.open_resource('GPIB0::22::INSTR')
DMM.read_termination = "\r\n"
DMM.timeout = 5000

### Data collection functions

In [7]:
def get_data(t_total, do_tqdm=True, dt=1):
    # collect data
    data = []
    t0 = time.time()
    if do_tqdm:
        pbar = tqdm(total=t_total)

    while time.time()-t0 < t_total:
        # get data from HV_control board
        data_str = HV_control.query('a')
        vals_arr = [(int(x)-32768) / 65535 * (2 * 3 * 4.096) for x in data_str.split(',')]

        data.append([time.time(), *vals_arr])
        if do_tqdm:
            pbar.update(int(time.time()-t0)+1 - pbar.n)
        time.sleep(dt)

    if do_tqdm:
        pbar.close()
    data = np.array(data)
    
    return data
    
def data_to_HDF(run_name, t_total, dt, fname="test.hdf", overwrite=False, do_tqdm=False, comment=""):
    # check that a dataset with this run_name doesn't already exist
    try:
        with h5py.File(fname, 'a') as f:
            if run_name in f.keys():
                if overwrite:
                    del f[run_name]
                else:
                    print("ERROR: Run name already exists.")
                    return
    except OSError:
        pass
    
    # get data
    data = get_data(t_total, do_tqdm, dt)
    
    # save to HDF
    with h5py.File(fname, 'a') as f:
        dset = f.create_dataset(run_name, data=data)
        dset.attrs['timestamp'] = time.time()
        dset.attrs['overwritten'] = str(overwrite)
        dset.attrs['t_total'] = t_total
        dset.attrs['dt'] = dt
        dset.attrs['comment'] = comment

In [8]:
def ramp(V_arr, dt=.1, printing=True, beep=False, do_tqdm=True):
    # whether to display a progress bar
    if do_tqdm:
        tqdm_fn = tqdm
    else:
        tqdm_fn = lambda x:x
        
    for V_set in tqdm_fn(V_arr):
        if beep:
            winsound.Beep(1000, 500)
        if printing:
            print(f"{int(V_set)} = {V_set/59055*30:.2f} kV     ", end="\r")
        HV_control.write(f'd{int(V_set)}')
        time.sleep(dt)

### Testing

In [51]:
HV_control.write('p0');
HV_control.write('r1');
HV_control.write('h1');
HV_control.write('d0');

In [54]:
HV_control.write(f'd{int(59055 / 30 * 0)}');

In [55]:
HV_control.write('h0');
HV_control.write(f'd{int(59055 / 30 * 0)}');

In [56]:
ramp(59055 / 30 * np.linspace(0, 10, 250))

  0%|          | 0/250 [00:00<?, ?it/s]

In [58]:
for i in tqdm(range(999)):
    data_to_HDF(
        run_name  = f"A-{i}",
        t_total   = 60,
        dt        = 1,
        fname     = "negative_10may23.hdf",
        overwrite = True,
        do_tqdm   = False,
        comment   = f"p1, h1, r1",
    )

  0%|          | 0/999 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [59]:
ramp(59055 / 30 * np.linspace(10, 0, 100))

  0%|          | 0/100 [00:00<?, ?it/s]

In [62]:
HV_control.write("p1")
HV_control.write("h1")
HV_control.write("r1")

for i,V_set in enumerate(tqdm([*np.linspace(0, 6553, 9), 0])):
    HV_control.write(f'd{int(V_set)}')
    
    data_to_HDF(
        run_name  = f"C-{i}",
        t_total   = 20,
        dt        = 1,
        fname     = "HV_test.hdf",
        overwrite = False,
        do_tqdm   = False,
        comment   = f"V_set={int(V_set)}, p1, h1, r1",
    )

  0%|          | 0/10 [00:00<?, ?it/s]

In [63]:
HV_control.write("p0")
HV_control.write("h1")
HV_control.write("r1")

for i,V_set in enumerate(tqdm([*np.linspace(0, 6553, 9), 0])):
    HV_control.write(f'd{int(V_set)}')
    
    data_to_HDF(
        run_name  = f"D-{i}",
        t_total   = 20,
        dt        = 1,
        fname     = "HV_test.hdf",
        overwrite = False,
        do_tqdm   = False,
        comment   = f"V_set={int(V_set)}, p0, h1, r1",
    )

  0%|          | 0/10 [00:00<?, ?it/s]

In [64]:
HV_control.write("p1")
HV_control.write("h0")
HV_control.write("r1")

for i,V_set in enumerate(tqdm([*np.linspace(0, 6553, 9), 0])):
    HV_control.write(f'd{int(V_set)}')
    
    data_to_HDF(
        run_name  = f"E-{i}",
        t_total   = 20,
        dt        = 1,
        fname     = "HV_test.hdf",
        overwrite = False,
        do_tqdm   = False,
        comment   = f"V_set={int(V_set)}, p0, h0, r1",
    )

  0%|          | 0/10 [00:00<?, ?it/s]

In [74]:
HV_control.write("p1")
HV_control.write("h1")
HV_control.write("r1")

for i,V_set in enumerate(tqdm([*np.linspace(0, 6553, 3), 0])):
    HV_control.write(f'd{int(V_set)}')
    
    data_to_HDF(
        run_name  = f"G-{i}",
        t_total   = 20,
        dt        = 1,
        fname     = "HV_test.hdf",
        overwrite = False,
        do_tqdm   = False,
        comment   = f"V_set={int(V_set)}, p0, h0, r1",
    )

  0%|          | 0/4 [00:00<?, ?it/s]

In [20]:
HV_control.write("d0");

In [142]:
HV_control.write("h0");

In [141]:
HV_control.write("h1");

In [138]:
HV_control.write("p0");

In [139]:
HV_control.write("p1");

In [55]:
data_str = HV_control.query('a')
[(int(x)-32768) / 65535 * (2 * 3 * 4.096) for x in data_str.split(',')]

[12.287812497138933,
 12.287812497138933,
 0.01762526894026093,
 0.007500114442664225,
 -4.317815884641794]

In [126]:
HV_control.write("p0")
HV_control.write("h1")
HV_control.write("r1")

for i,V_set in enumerate(tqdm([*np.linspace(0, 6553, 3), 0])):
    HV_control.write(f'd{int(V_set)}')
    
    data_to_HDF(
        run_name  = f"I-{i}",
        t_total   = 20,
        dt        = 1,
        fname     = "HV_test.hdf",
        overwrite = False,
        do_tqdm   = False,
        comment   = f"V_set={int(V_set)}, p0, h1, r1",
    )

  0%|          | 0/4 [00:00<?, ?it/s]

In [130]:
HV_control.write("p0")
HV_control.write("h0")
HV_control.write("r1")

for i,V_set in enumerate(tqdm([*np.linspace(0, 6553, 3), 0])):
    HV_control.write(f'd{int(V_set)}')
    
    data_to_HDF(
        run_name  = f"J-{i}",
        t_total   = 20,
        dt        = 1,
        fname     = "HV_test.hdf",
        overwrite = False,
        do_tqdm   = False,
        comment   = f"V_set={int(V_set)}",
    )

  0%|          | 0/4 [00:00<?, ?it/s]

In [134]:
HV_control.write("p0")
HV_control.write("h0")
HV_control.write("r1")

for i,V_set in enumerate(tqdm([*np.linspace(0, 6553, 3), 0])):
    HV_control.write(f'd{int(V_set)}')
    
    data_to_HDF(
        run_name  = f"K-{i}",
        t_total   = 20,
        dt        = 1,
        fname     = "HV_test.hdf",
        overwrite = False,
        do_tqdm   = False,
        comment   = f"V_set={int(V_set)}",
    )

  0%|          | 0/4 [00:00<?, ?it/s]

In [135]:
HV_control.write("p0")
HV_control.write("h1")
HV_control.write("r1")

for i,V_set in enumerate(tqdm([*np.linspace(0, 6553, 3), 0])):
    HV_control.write(f'd{int(V_set)}')
    
    data_to_HDF(
        run_name  = f"L-{i}",
        t_total   = 20,
        dt        = 1,
        fname     = "HV_test.hdf",
        overwrite = False,
        do_tqdm   = False,
        comment   = f"V_set={int(V_set)}",
    )

  0%|          | 0/4 [00:00<?, ?it/s]

In [137]:
HV_control.write("p1")
HV_control.write("h1")
HV_control.write("r1")

for i,V_set in enumerate(tqdm([*np.linspace(0, 6553, 3), 0])):
    HV_control.write(f'd{int(V_set)}')
    
    data_to_HDF(
        run_name  = f"M-{i}",
        t_total   = 20,
        dt        = 1,
        fname     = "HV_test.hdf",
        overwrite = False,
        do_tqdm   = False,
        comment   = f"V_set={int(V_set)}",
    )

  0%|          | 0/4 [00:00<?, ?it/s]

In [143]:
HV_control.write("p1")
HV_control.write("h0")
HV_control.write("r1")

for i,V_set in enumerate(tqdm([*np.linspace(0, 6553, 3), 0])):
    HV_control.write(f'd{int(V_set)}')
    
    data_to_HDF(
        run_name  = f"N-{i}",
        t_total   = 20,
        dt        = 1,
        fname     = "HV_test.hdf",
        overwrite = False,
        do_tqdm   = False,
        comment   = f"V_set={int(V_set)}",
    )

  0%|          | 0/4 [00:00<?, ?it/s]

In [144]:
HV_control.write("p1")
HV_control.write("h1")
HV_control.write("r1")

for i,V_set in enumerate(tqdm([*np.linspace(0, 6553, 3), 0])):
    HV_control.write(f'd{int(V_set)}')
    
    data_to_HDF(
        run_name  = f"O-{i}",
        t_total   = 20,
        dt        = 1,
        fname     = "HV_test.hdf",
        overwrite = False,
        do_tqdm   = False,
        comment   = f"V_set={int(V_set)}",
    )

  0%|          | 0/4 [00:00<?, ?it/s]

In [184]:
HV_control.write("p1")
HV_control.write("h1")
HV_control.write("r1")

for i,V_set in enumerate(tqdm([*np.linspace(0, 20000, 10), 0])):
    HV_control.write(f'd{int(V_set)}')
    
    data_to_HDF(
        run_name  = f"P-{i}",
        t_total   = 20,
        dt        = 1,
        fname     = "HV_test.hdf",
        overwrite = True,
        do_tqdm   = False,
        comment   = f"V_set={int(V_set)}",
    )
    
HV_control.write("h1");

  0%|          | 0/11 [00:00<?, ?it/s]

In [186]:
HV_control.write("p0")
HV_control.write("h1")
HV_control.write("r1")

for i,V_set in enumerate(tqdm([*np.linspace(0, 20000, 10), 0])):
    HV_control.write(f'd{int(V_set)}')
    
    data_to_HDF(
        run_name  = f"Q-{i}",
        t_total   = 20,
        dt        = 1,
        fname     = "HV_test.hdf",
        overwrite = True,
        do_tqdm   = False,
        comment   = f"V_set={int(V_set)}",
    )
    
HV_control.write("h1");

  0%|          | 0/11 [00:00<?, ?it/s]

In [187]:
HV_control.write("p1")
HV_control.write("h0")
HV_control.write("r1")

for i,V_set in enumerate(tqdm([*np.linspace(0, 20000, 10), 0])):
    HV_control.write(f'd{int(V_set)}')
    
    data_to_HDF(
        run_name  = f"R-{i}",
        t_total   = 20,
        dt        = 1,
        fname     = "HV_test.hdf",
        overwrite = True,
        do_tqdm   = False,
        comment   = f"V_set={int(V_set)}",
    )
    
HV_control.write("h1");

  0%|          | 0/11 [00:00<?, ?it/s]

In [183]:
HV_control.write("p0")
HV_control.write("h0")
HV_control.write("r1")

for i,V_set in enumerate(tqdm([*np.linspace(0, 20000, 10), 0])):
    HV_control.write(f'd{int(V_set)}')
    
    data_to_HDF(
        run_name  = f"S-{i}",
        t_total   = 20,
        dt        = 1,
        fname     = "HV_test.hdf",
        overwrite = True,
        do_tqdm   = False,
        comment   = f"V_set={int(V_set)}",
    )
    
HV_control.write("h1");

  0%|          | 0/11 [00:00<?, ?it/s]

In [172]:
HV_control.write("p0");

In [ ]:
HV_control.write("p1");

In [182]:
HV_control.write("h0");

In [181]:
HV_control.write("h1");